# Test des fonctionnalités DP_impose et K regul

In [ ]:
from trustutils import run 
run.introduction("A. Gerschenfeld, Y. Gorsse")
run.description("")
run.TRUST_parameters()

In [ ]:
from trustutils import run
from string import Template
import os, sys
run.useMEDCoupling()
import medcoupling as mc

run.reset()
meshes = {
    "Cartesian": ["PolyMAC", "PolyMAC_P0"], 
}

# Trigger the copy of the src directory into the build directory:
run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)
origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)
for m, dis in meshes.items():
    for d in dis:
        os.system(f"mkdir -p {m}/{d}")
        with open("jdd.data", "r") as file: filedata = Template(file.read())
        result = filedata.substitute({"dis" : d})
        with open(f"{m}/{d}/jdd.data", "w") as file: file.write(result)
        if not d.startswith("Poly"):
            #mfum = mc.MEDFileUMesh(meshname)
            #mfum.getMeshAtLevel(0).unPolyze()
            #mfum.write(f"{m}/{d}/mesh.med", 2)
            os.system(f"cp mesh_hexa.med {m}/{d}/mesh.med")
        else: os.system(f"cp mesh_poly.med {m}/{d}/mesh.med")
        run.addCase(f"{m}/{d}", "jdd.data")

os.chdir(origin)

run.printCases()
run.runCases()

## Description
Domaine fermé avec une frontiere interne (en gris sur la figure), et deux surfaces permettant d'imposer les termes source testés. En bleu, la régulation sur le débit, et en orange le DP imposé. Une circulation du fluide apparait, et le débit généré se régule sur la valeur cible.

![](src/domaine.png)

In [ ]:
run.executeCommand("paste PolyMAC/jdd_P1.son PolyMAC/jdd_P2.son > PolyMAC/jdd_P1P2.son")
run.executeCommand("paste PolyMAC_P0/jdd_P1.son PolyMAC_P0/jdd_P2.son > PolyMAC_P0/jdd_P1P2.son")

In [ ]:
from trustutils.jupyter import plot
import numpy as np

dis = ["PolyMAC", "PolyMAC_P0"]
Graphique=plot.Graph(nX=2,nY=len(dis))

for i, d in enumerate(dis):
    Graphique.addPlot([0, i], d)
    data = plot.loadText(f"Cartesian/{d}/jdd_pb_K_surf1.out", skiprows=3)
    Graphique.add(data[0], data[1] ,label="K")
    Graphique.add(data[0],-data[2] ,label="debit calcule")
    Graphique.add(data[0], data[3] ,label="debit cible")
    Graphique.label("Time", "Delta de pression")

    Graphique.addPlot([1, i], d)
    data = plot.loadText(f"Cartesian/{d}/jdd_pb_DP_surf2.out", skiprows=3)
    Graphique.add(data[0], data[1], label="DP impose")
    data1 = plot.loadText(f"Cartesian/{d}/jdd_P1.son", skiprows=5)
    data2 = plot.loadText(f"Cartesian/{d}/jdd_P2.son", skiprows=5)

    Graphique.add(data[0], data1[1] - data2[1],label="DP mesure")

Graphique.visu()